### Create yield and price stats (AVG & GBM parameters)

In [7]:
#### Append GBM parameters

from statistics import mean,stdev 
import numpy as np
import json
import numpy as np

# Your JSON data
json_file_path = "./environment_maincrops/data/maincrops.json"

# Open and read the JSON file
with open(json_file_path, "r") as json_file:
    # Parse the JSON data into a Python dictionary
    data = json.load(json_file)

# # Function to calculate average and standard deviation
def calculate_stats(values):
    avg = mean(values)
    std = stdev(values)
    maximum = max(values)
    return avg, std, maximum


# # # Function to calculate average and standard deviation
def calculate_stats_gbm(values):
    avg = mean(values)
    std = stdev(values)
    maximum = max(values)

    returns = np.diff(values) / values[:-1]

    # Calculate the average annual return (drift)
    avg_return = mean(returns)

    # Calculate annual volatility (sigma)
    volatility = stdev(returns)
    return avg, std, maximum, avg_return, volatility



kosten_keys = ["Saatgut","Duenger N","Duenger P","Duenger K","Sonstiges"]
# # # Iterate through each crop
for crop_key, crop_data in data.items():
    print(crop_key)

    # Ertrag-Stats
    ertrag_data = crop_data["Ertrag"]
    
    # Iterate through each region under "Ertrag"
    for region_key, region_data in ertrag_data.items():
        values = region_data["Values"]
        
        # Calculate average and standard deviation
        avg, std, maximum = calculate_stats(values)
        
        # Add "avg" and "std" to the region_data
        region_data["avg"] = np.round(avg,3).astype(float)
        region_data["std"] = np.round(std,3).astype(float)
        region_data["max"] = np.round(maximum,3).astype(float)

    # Verkaufspreis GBM Stats
    verkaufspreis = crop_data["Verkaufspreis"]
    values = verkaufspreis["Values"]
        
    # Calculate average and standard deviation
    avg, std, maximum, avg_return, volatility = calculate_stats_gbm(values)
        
    # Add "avg" and "std" to the region_data
    verkaufspreis["avg"] = np.round(avg,3).astype(float)
    verkaufspreis["std"] = np.round(std,3).astype(float)
    verkaufspreis["max"] = np.round(maximum,3).astype(float)
    verkaufspreis["avg_gbm"] = np.round(avg_return,6).astype(float)
    verkaufspreis["std_gbm"] = np.round(volatility,6).astype(float)


    # Kosten GBM Stats
    kosten = crop_data["Kosten"]
    for key in kosten_keys:
        values = kosten[key]["Values"]
        avg, std, maximum, avg_return, volatility = calculate_stats_gbm(values)
        kosten[key]["avg"] = np.round(avg,3).astype(float)
        kosten[key]["std"] = np.round(std,3).astype(float)
        kosten[key]["max"] = np.round(maximum,3).astype(float)
        kosten[key]["avg_gbm"] = np.round(avg_return,6).astype(float)
        kosten[key]["std_gbm"] = np.round(volatility,6).astype(float)

# Dump the dictionary to a JSON file with ensure_ascii=False
with open("./environment_maincrops/data/maincrops_updated.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False)

KLEEGRAS
LUZERNE
ACKERBOHNE
KOERNERERBSE
SOJABOHNE
FUTTERLUPINE
WINTERWEIZEN
SOMMERWEIZEN
WINTERHARTWEIZEN
WINTERDINKEL
WINTERTRITICALE
WINTERROGGEN
WINTERGERSTE
SOMMERGERSTE
SOMMERHAFER
HIRSE
SILOMAIS
KOERNERMAIS
ZUCKERRUEBEN
KARTOFFELN
WINTERRAPS
SONNENBLUMEN
OELKUERBIS
GRASSILAGE


### Kolbe matrix as dictionary

In [38]:
import pandas as pd
import pprint

class CustomPrettyPrinter(pprint.PrettyPrinter):
    def format(self, object, context, maxlevels, level):
        if isinstance(object, list):
            # Customize how lists are formatted
            if len(object) > 5:  # Adjust this threshold as needed
                return self._format_multiline_list(object, context, maxlevels, level)
            else:
                return super().format(object, context, maxlevels, level)
        return super().format(object, context, maxlevels, level)

    def _format_multiline_list(self, object, context, maxlevels, level):
        output = "[" + ", ".join(map(repr, object)) + "]"
        return output, True, False

pp = CustomPrettyPrinter(width=160)
# Read the Excel file into a DataFrame
excel_file = './environment_maincrops/data/kolbe.xlsx'  # Replace with the path to your Excel file
df = pd.read_excel(excel_file, header=1, index_col=0)

# Create a dictionary to store the matrix
matrix_dict = {"PRECROP": {}}

# Map plant names to integers
plant_mapping = {
    'KLEEGRAS': 0,
    'LUZERNE': 1,
    'ACKERBOHNE': 2,
    'KOERNERERBSE': 3,
    'SOJABOHNE': 4,
    'FUTTERLUPINE': 5,
    'WINTERWEIZEN': 6,
    'SOMMERWEIZEN': 7,
    'WINTERHARTWEIZEN': 8,
    'WINTERDINKEL': 9,
    'WINTERTRITICALE': 10,
    'WINTERROGGEN': 11,
    'WINTERGERSTE': 12,
    'SOMMERGERSTE': 13,
    'SOMMERHAFER': 14,
    'HIRSE': 15,
    'SILOMAIS': 16,
    'KOERNERMAIS': 17,
    'ZUCKERRUEBEN': 18,
    'KARTOFFELN': 19,
    'WINTERRAPS': 20,
    'SONNENBLUMEN': 21,
    'OELKUERBIS': 22
}

# Iterate through the rows of the DataFrame and populate the matrix_dict
for row_name, row_values in df.iterrows():
    row_index = plant_mapping.get(row_name)
    matrix_dict["PRECROP"][row_index] = []
    first_col = True
    for col_name, col_value in row_values.items():
        if first_col:
            first_col = False
            continue
        col_index = plant_mapping.get(col_name)
        matrix_dict["PRECROP"][row_index].append(col_value)
    if row_name == "OELKUERBIS":
        break

# Print or use the matrix_dict as needed
pp.pprint(matrix_dict)

{'PRECROP': {0: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, -2.0, -1.0],
             1: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, -1.0, -1.0],
             2: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, -2.0, -1.0, 2.0],
             3: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, -1.0, 2.0],
             4: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, -2.0, -1.0, -1.0],
             5: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, -2.0, -1.0, 2.0],
             6: [1.0, 2.0, -1.0, -1.0, 2.0, -1.0, -2.0, -2.0, -2.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, -1.0, 1.0, 2.0],
    

### Create dictionary for crop breaks

In [6]:
import pprint
import pandas as pd

plant_mapping = {
    'KLEEGRAS': 0,
    'LUZERNE': 1,
    'ACKERBOHNE': 2,
    'KOERNERERBSE': 3,
    'SOJABOHNE': 4,
    'FUTTERLUPINE': 5,
    'WINTERWEIZEN': 6,
    'SOMMERWEIZEN': 7,
    'WINTERHARTWEIZEN': 8,
    'WINTERDINKEL': 9,
    'WINTERTRITICALE': 10,
    'WINTERROGGEN': 11,
    'WINTERGERSTE': 12,
    'SOMMERGERSTE': 13,
    'SOMMERHAFER': 14,
    'HIRSE': 15,
    'SILOMAIS': 16,
    'KOERNERMAIS': 17,
    'ZUCKERRUEBEN': 18,
    'KARTOFFELN': 19,
    'WINTERRAPS': 20,
    'SONNENBLUMEN': 21,
    'OELKUERBIS': 22
}


pp = pprint.PrettyPrinter(indent=2)

# Create the dictionary
result = {}

excel_file = './environment_maincrops/data/kolbe.xlsx'  # Replace with the path to your Excel file
df = pd.read_excel(excel_file,sheet_name = 1, header=0, index_col=1)

# Iterate through rows
for index, row in df.iterrows():
    if pd.isna(index):
        break
    # print(plant_mapping[index])
    crop_name = index
    ap_groups = []
    mf_groups = []

    # Iterate through columns starting with 'AP' or 'MF'
    for column_name, value in row.items():
        if column_name in ["Familie","AP Pflanze"]:
            continue
        if column_name.startswith('AP') and not pd.isna(value):
            ap_groups.append(" ".join(column_name.split(" ")[1:]))
        elif column_name.startswith('MF') and not pd.isna(value):
            mf_groups.append(" ".join(column_name.split(" ")[1:]))

    # Add the data to the dictionary
    crop_data = {
        "crop":crop_name,
        "AP": row['AP Pflanze'],
        "AP Groups": ap_groups,
        "MF Groups": mf_groups,
    }
    result[plant_mapping[index]] = crop_data

# Print the resulting dictionary
pp.pprint(result)

KeyError: 'KÖRNERERBSE'

# Getting information for Copilot instructions

## Properties Filter

In [8]:
import pprint
import json
from environment_maincrops.data.mappings import date_mapping_rev, crop_mapping_german_rev
from environment_maincrops.data.cropbreaks import cropbreaks, mf_groups, ap_groups
pp = pprint.PrettyPrinter(indent=2)

json_file_path = ".\environment_maincrops\data\maincrop_properties.json"
with open(json_file_path, "r") as json_file:
    # Parse the JSON data into a Python dictionary
    data = json.load(json_file)

pdict = {} 
for action,properties in data.items():
    action = crop_mapping_german_rev[action]
    properties["drought_resistance"] = properties["drought resistance"]
    properties["earliest_harvest"] = date_mapping_rev[properties["earliest harvest"]]
    properties["latest_sowing"] = date_mapping_rev[properties["latest sowing"]]
    properties["humus_equivalent"] = int(properties["humus equivalent"])
    gt_list = []
    for value in properties["ground type"]:
        if value == "light":
            gt_list.append(-1)
        elif value == "medium":
            gt_list.append(0)
        else:
            gt_list.append(1)
    properties["ground_type"] = gt_list
    properties.pop("humus effect")
    properties.pop("humus equivalent")
    properties.pop("drought resistance")
    properties.pop("postdelivery")
    properties.pop("earliest harvest")
    properties.pop("latest sowing")
    properties.pop("ground type")
    pdict[action] = properties

# pp.pprint(pdict)

for action, properties in pdict.items():
    for pname, value in properties.items():
        if isinstance(value, list):
            for elem in value:
                print(f"property({action}, {pname}, {elem}).")
        else:
            print(f"property({action}, {pname}, {value}).")


pdict = {} 
for action,properties in cropbreaks.items():
    AP = properties["AP"]
    for i in range(1,AP+1):
        print(f"cropbreak({action}, single, -{i}).")

replace_dict = {
    "ü":"ue",
    "ä":"ae",
    "ö":"oe",
    "/":""
}

def replace_from_dict(s, replace_dict):
    for key, value in replace_dict.items():
        s = s.replace(key, value)
    return s

pdict = {} 
for action,properties in cropbreaks.items():
    AP_groups = properties["AP Groups"]
    if len(AP_groups) == 0:
        continue
    elif len(AP_groups) == 1:
        print(f"apgroups({action}, {replace_from_dict(AP_groups[0].lower(),replace_dict).lower()}).")
    else:
        AP_groups = [replace_from_dict(ap_group.lower(),replace_dict) for ap_group in AP_groups]
        AP_groups = ";".join(AP_groups)
        print(f"apgroups({action}, ({AP_groups})).")
    

for ap_group, crops in ap_groups["group_indices"].items():
    ap = ap_groups["min_breaks"][ap_group]
    if ap > 1:
        aps = f"{-ap}..-1"
    else:
        aps = f"-1"
    crops = [str(crop).lower() for crop in crops]
    crop_string = ';'.join(crops)
    print(f"ap_group_block({replace_from_dict(ap_group.lower(),replace_dict)}, ({crop_string}), {aps}).")

# pp.pprint(pdict)




property(0, summercrop, 0).
property(0, drought_resistance, 0).
property(0, earliest_harvest, 13).
property(0, latest_sowing, 25).
property(0, humus_equivalent, 1300).
property(0, ground_type, 0).
property(1, summercrop, 0).
property(1, drought_resistance, 1).
property(1, earliest_harvest, 13).
property(1, latest_sowing, 22).
property(1, humus_equivalent, 1300).
property(1, ground_type, -1).
property(1, ground_type, 0).
property(2, summercrop, 1).
property(2, drought_resistance, 0).
property(2, earliest_harvest, 19).
property(2, latest_sowing, 9).
property(2, humus_equivalent, 160).
property(2, ground_type, 0).
property(2, ground_type, 1).
property(3, summercrop, 1).
property(3, drought_resistance, -1).
property(3, earliest_harvest, 19).
property(3, latest_sowing, 11).
property(3, humus_equivalent, 160).
property(3, ground_type, 0).
property(3, ground_type, 1).
property(4, summercrop, 1).
property(4, drought_resistance, -1).
property(4, earliest_harvest, 25).
property(4, latest_sowing,

In [5]:
import clingo 

# Define the Clingo program as a string
program = """
% Define the input data as facts
action(0..23).
property(0, summercrop, 0).
property(0, drought_resistance, 0).
property(0, earliest_harvest, 13).
property(0, latest_sowing, 25).
property(0, ground_type, 0).
property(1, summercrop, 0).
property(1, drought_resistance, 1).
property(1, earliest_harvest, 13).
property(1, latest_sowing, 22).
property(1, ground_type, -1).
property(1, ground_type, 0).
property(2, summercrop, 1).
property(2, drought_resistance, 0).
property(2, earliest_harvest, 19).
property(2, latest_sowing, 9).
property(2, ground_type, 0).
property(2, ground_type, 1).
property(3, summercrop, 1).
property(3, drought_resistance, -1).
property(3, earliest_harvest, 19).
property(3, latest_sowing, 11).
property(3, ground_type, 0).
property(3, ground_type, 1).
property(4, summercrop, 1).
property(4, drought_resistance, -1).
property(4, earliest_harvest, 25).
property(4, latest_sowing, 12).
property(4, ground_type, 0).
property(5, summercrop, 1).
property(5, drought_resistance, 0).
property(5, earliest_harvest, 23).
property(5, latest_sowing, 9).
property(5, ground_type, 0).
property(6, summercrop, 0).
property(6, drought_resistance, 0).
property(6, earliest_harvest, 19).
property(6, latest_sowing, 31).
property(6, ground_type, 0).
property(6, ground_type, 1).
property(7, summercrop, 1).
property(7, drought_resistance, -1).
property(7, earliest_harvest, 20).
property(7, latest_sowing, 8).
property(7, ground_type, 0).
property(7, ground_type, 1).
property(8, summercrop, 0).
property(8, drought_resistance, 0).
property(8, earliest_harvest, 19).
property(8, latest_sowing, 28).
property(8, ground_type, 0).
property(8, ground_type, 1).
property(9, summercrop, 0).
property(9, drought_resistance, 0).
property(9, earliest_harvest, 19).
property(9, latest_sowing, 31).
property(9, ground_type, 0).
property(9, ground_type, 1).
property(10, summercrop, 0).
property(10, drought_resistance, 0).
property(10, earliest_harvest, 19).
property(10, latest_sowing, 28).
property(10, ground_type, 0).
property(10, ground_type, 1).
property(11, summercrop, 0).
property(11, drought_resistance, 1).
property(11, earliest_harvest, 22).
property(11, latest_sowing, 28).
property(11, ground_type, -1).
property(11, ground_type, 0).
property(12, summercrop, 0).
property(12, drought_resistance, 1).
property(12, earliest_harvest, 19).
property(12, latest_sowing, 28).
property(12, ground_type, -1).
property(12, ground_type, 0).
property(13, summercrop, 1).
property(13, drought_resistance, -1).
property(13, earliest_harvest, 20).
property(13, latest_sowing, 9).
property(13, ground_type, 0).
property(14, summercrop, 1).
property(14, drought_resistance, -1).
property(14, earliest_harvest, 21).
property(14, latest_sowing, 8).
property(14, ground_type, 0).
property(15, summercrop, 1).
property(15, drought_resistance, 1).
property(15, earliest_harvest, 27).
property(15, latest_sowing, 15).
property(15, ground_type, -1).
property(15, ground_type, 0).
property(16, summercrop, 1).
property(16, drought_resistance, -1).
property(16, earliest_harvest, 25).
property(16, latest_sowing, 13).
property(16, ground_type, 0).
property(16, ground_type, 1).
property(17, summercrop, 1).
property(17, drought_resistance, -1).
property(17, earliest_harvest, 24).
property(17, latest_sowing, 13).
property(17, ground_type, 0).
property(17, ground_type, 1).
property(18, summercrop, 1).
property(18, drought_resistance, 0).
property(18, earliest_harvest, 24).
property(18, latest_sowing, 10).
property(18, ground_type, 0).
property(19, summercrop, 1).
property(19, drought_resistance, -1).
property(19, earliest_harvest, 22).
property(19, latest_sowing, 14).
property(19, ground_type, -1).
property(19, ground_type, 0).
property(20, summercrop, 0).
property(20, drought_resistance, 0).
property(20, earliest_harvest, 19).
property(20, latest_sowing, 24).
property(20, ground_type, 0).
property(20, ground_type, 1).
property(21, summercrop, 1).
property(21, drought_resistance, -1).
property(21, earliest_harvest, 23).
property(21, latest_sowing, 11).
property(21, ground_type, -1).
property(21, ground_type, 0).
property(22, summercrop, 1).
property(22, drought_resistance, -1).
property(22, earliest_harvest, 24).
property(22, latest_sowing, 13).
property(22, ground_type, -1).
property(22, ground_type, 0).
property(23, summercrop, 0).
property(23, drought_resistance, 1).
property(23, earliest_harvest, 13).
property(23, latest_sowing, 25).
property(23, humus_equivalent, 1300).
property(23, ground_type, -1).
property(23, ground_type, 0).
property(23, ground_type, 1).
cropbreak(0, single, -1).
cropbreak(0, single, -2).
cropbreak(0, single, -3).
cropbreak(0, single, -4).
cropbreak(1, single, -1).
cropbreak(1, single, -2).
cropbreak(1, single, -3).
cropbreak(1, single, -4).
cropbreak(1, single, -5).
cropbreak(2, single, -1).
cropbreak(2, single, -2).
cropbreak(2, single, -3).
cropbreak(3, single, -1).
cropbreak(3, single, -2).
cropbreak(3, single, -3).
cropbreak(3, single, -4).
cropbreak(3, single, -5).
cropbreak(4, single, -1).
cropbreak(4, single, -2).
cropbreak(4, single, -3).
cropbreak(5, single, -1).
cropbreak(5, single, -2).
cropbreak(5, single, -3).
cropbreak(5, single, -4).
cropbreak(6, single, -1).
cropbreak(6, single, -2).
cropbreak(7, single, -1).
cropbreak(7, single, -2).
cropbreak(8, single, -1).
cropbreak(8, single, -2).
cropbreak(9, single, -1).
cropbreak(9, single, -2).
cropbreak(10, single, -1).
cropbreak(10, single, -2).
cropbreak(11, single, -1).
cropbreak(12, single, -1).
cropbreak(12, single, -2).
cropbreak(13, single, -1).
cropbreak(13, single, -2).
cropbreak(14, single, -1).
cropbreak(14, single, -2).
cropbreak(14, single, -3).
cropbreak(14, single, -4).
cropbreak(15, single, -1).
cropbreak(15, single, -2).
cropbreak(16, single, -1).
cropbreak(17, single, -1).
cropbreak(18, single, -1).
cropbreak(18, single, -2).
cropbreak(18, single, -3).
cropbreak(18, single, -4).
cropbreak(19, single, -1).
cropbreak(19, single, -2).
cropbreak(19, single, -3).
cropbreak(20, single, -1).
cropbreak(20, single, -2).
cropbreak(20, single, -3).
cropbreak(21, single, -1).
cropbreak(21, single, -2).
cropbreak(21, single, -3).
cropbreak(22, single, -1).
cropbreak(22, single, -2).

ground_type_info(-1).
drywet_info(0).
previous_actions_info(-5,1).
previous_actions_info(-4,2).
previous_actions_info(-3,15).
previous_actions_info(-2,4).
previous_actions_info(-1,5).
week_info(0).

% Define
blocked_by_previous(A) :- action(A), previous_actions_info(X,A), cropbreak(A, single, X).
blocked_by_week(A) :- action(A), property(A, earliest_harvest, EH), week_info(W),  W > EH-1, property(A, summercrop, SC), SC == 0.
allowed_by_drywet(A) :- action(A), property(A, drought_resistance, DR), drywet_info(DW),  DR > -1, DW == 0.
allowed_by_drywet(A) :- action(A), drywet_info(DW), DW == 1.
allowed_by_groundtype(A) :- action(A), ground_type_info(GT), property(A, ground_type, GT).

immediate_candidate(A) :- action(A), not blocked_by_previous(A), not blocked_by_week(A), allowed_by_drywet(A), allowed_by_groundtype(A).

% Define the output predicate
#show immediate_candidate/1.

"""
# Load the Clingo program
control = clingo.Control()
control.add("base", [], program)
control.ground([("base", [])])

# Solve the program and print the immediate candidate actions
solutions = []
def on_model(model):
    solutions.append(model.symbols(shown=True))

control.solve(on_model=on_model)
for solution in solutions:
    for symbol in solution:
        if symbol.name == "immediate_candidate":
            print("candidate:", symbol.arguments[0])
        if symbol.name == "blocked":
            print("candidate:", symbol.arguments[0])

candidate: 11
candidate: 12
candidate: 15
candidate: 23


## AP Groups Filter

In [4]:
import clingo 

# Define the Clingo program as a string
program = """
% Define the input data as facts
action(0..22).
apgroups(0, (blatt;fl;l)).
apgroups(1, (blatt;fl;l)).
apgroups(2, (blatt;l)).
apgroups(3, (blatt;l)).
apgroups(4, (blatt;l)).
apgroups(5, (blatt;l)).
apgroups(6, (weizendinkeltriticale;weizen)).
apgroups(7, (weizendinkeltriticale;weizen)).
apgroups(8, (weizendinkeltriticale;weizen)).
apgroups(9, weizendinkeltriticale).
apgroups(10, weizendinkeltriticale).
apgroups(12, gerste).
apgroups(13, gerste).
apgroups(16, (blatt;mais)).
apgroups(17, mais).
apgroups(18, (blatt;ruebenkruziferen)).
apgroups(19, blatt).
apgroups(20, (rapssonnenblume;ruebenkruziferen)).
apgroups(21, rapssonnenblume).
ap_group_block(blatt, (0;1;2;3;4;5;16;18;19), -1).
ap_group_block(fl, (0;1), -5..-1).
ap_group_block(l, (0;1;2;3;4;5), -4..-1).
ap_group_block(weizendinkeltriticale, (6;7;8;9;10), -1).
ap_group_block(rapssonnenblume, (20;21), -2..-1).
ap_group_block(mais, (16;17), -1).
ap_group_block(ruebenkruziferen, (18;20), -3..-1).
ap_group_block(weizen, (18;20), -2..-1).
ap_group_block(gerste, (12;13), -2..-1).

previous_actions_info(-5,1).
previous_actions_info(-4,2).
previous_actions_info(-3,15).
previous_actions_info(-2,4).
previous_actions_info(-1,5).


% Define
blocked_by_ap_group(A) :- apgroups(A,APG), previous_actions_info(X,Y), ap_group_block(APG, Y, X).


immediate_candidate(A) :- action(A), not blocked_by_ap_group(A).

% Define the output predicate
#show immediate_candidate/1.

"""


# Load the Clingo program
control = clingo.Control()
control.add("base", [], program)
control.ground([("base", [])])

# Solve the program and print the immediate candidate actions
solutions = []
def on_model(model):
    solutions.append(model.symbols(shown=True))

control.solve(on_model=on_model)
for solution in solutions:
    # print(solution)
    for symbol in solution:
        print(symbol.arguments[0])

6
7
8
9
10
11
12
13
14
15
17
20
21
22


## MF Group Filter

In [5]:
import pprint
import json
from environment_maincrops.data.mappings import date_mapping_rev, crop_mapping_german_rev
from environment_maincrops.data.cropbreaks import cropbreaks, mf_groups, ap_groups
pp = pprint.PrettyPrinter(indent=2)


replace_dict = {
    "ü":"ue",
    "ä":"ae",
    "ö":"oe",
    "/":"",
    "(":"",
    ")":"",
    " ":""
}

def replace_from_dict(s, replace_dict):
    for key, value in replace_dict.items():
        s = s.replace(key, value)
    return s

pdict = {} 
for action,properties in cropbreaks.items():
    MF_groups = properties["MF Groups"]
    if len(MF_groups) == 0:
        continue
    elif len(MF_groups) == 1:
        print(f"mfgroups({action}, {replace_from_dict(MF_groups[0].lower(),replace_dict)}).")
    else:
        MF_groups = [replace_from_dict(ap_group.lower(),replace_dict) for ap_group in MF_groups]
        MF_groups = ";".join(MF_groups)
        print(f"mfgroups({action}, ({MF_groups})).")
    

for mf_group, crops in mf_groups["group_indices"].items():
    max_count, total_count = mf_groups["max_frequencies"][mf_group]
    crops = [str(crop).lower() for crop in crops]
    crop_string = ';'.join(crops)
    print(f"mf_group_block({replace_from_dict(mf_group.lower(),replace_dict)}, ({crop_string}), -{total_count}..-1).")
    print(f"mf_group_block_max_count({replace_from_dict(mf_group.lower(),replace_dict)}, {max_count}).")

mfgroups(6, (ge;geohnemaishaferhirse;weizentriticale)).
mfgroups(7, (ge;geohnemaishaferhirse;weizentriticale)).
mfgroups(8, (ge;geohnemaishaferhirse;weizentriticale)).
mfgroups(9, (ge;geohnemaishaferhirse)).
mfgroups(10, (ge;geohnemaishaferhirse;weizentriticale)).
mfgroups(11, (ge;geohnemaishaferhirse)).
mfgroups(12, (ge;geohnemaishaferhirse)).
mfgroups(13, (ge;geohnemaishaferhirse)).
mfgroups(14, ge).
mfgroups(15, ge).
mfgroups(17, ge).
mf_group_block(ge, (6;7;8;9;10;11;12;13;14;15;17), -4..-1).
mf_group_block_max_count(ge, 3).
mf_group_block(geohnemaishaferhirse, (6;7;8;9;10;11;12;13), -3..-1).
mf_group_block_max_count(geohnemaishaferhirse, 2).
mf_group_block(weizentriticale, (6;7;8;10), -3..-1).
mf_group_block_max_count(weizentriticale, 1).


In [12]:
import clingo 

# Define the Clingo program as a string
program = """
% Define the input data as facts
previous_time(-5..-1).
action(0..23).
property(0, summercrop, 0).
property(0, drought_resistance, 0).
property(0, earliest_harvest, 13).
property(0, latest_sowing, 25).
property(0, humus_equivalent, 1300).
property(0, ground_type, 0).
property(1, summercrop, 0).
property(1, drought_resistance, 1).
property(1, earliest_harvest, 13).
property(1, latest_sowing, 22).
property(1, humus_equivalent, 1300).
property(1, ground_type, -1).
property(1, ground_type, 0).
property(2, summercrop, 1).
property(2, drought_resistance, 0).
property(2, earliest_harvest, 19).
property(2, latest_sowing, 9).
property(2, humus_equivalent, 160).
property(2, ground_type, 0).
property(2, ground_type, 1).
property(3, summercrop, 1).
property(3, drought_resistance, -1).
property(3, earliest_harvest, 19).
property(3, latest_sowing, 11).
property(3, humus_equivalent, 160).
property(3, ground_type, 0).
property(3, ground_type, 1).
property(4, summercrop, 1).
property(4, drought_resistance, -1).
property(4, earliest_harvest, 25).
property(4, latest_sowing, 12).
property(4, humus_equivalent, 160).
property(4, ground_type, 0).
property(5, summercrop, 1).
property(5, drought_resistance, 0).
property(5, earliest_harvest, 23).
property(5, latest_sowing, 9).
property(5, humus_equivalent, 160).
property(5, ground_type, 0).
property(6, summercrop, 0).
property(6, drought_resistance, 0).
property(6, earliest_harvest, 19).
property(6, latest_sowing, 31).
property(6, humus_equivalent, -400).
property(6, ground_type, 0).
property(6, ground_type, 1).
property(7, summercrop, 1).
property(7, drought_resistance, -1).
property(7, earliest_harvest, 20).
property(7, latest_sowing, 8).
property(7, humus_equivalent, -400).
property(7, ground_type, 0).
property(7, ground_type, 1).
property(8, summercrop, 0).
property(8, drought_resistance, 0).
property(8, earliest_harvest, 19).
property(8, latest_sowing, 28).
property(8, humus_equivalent, -400).
property(8, ground_type, 0).
property(8, ground_type, 1).
property(9, summercrop, 0).
property(9, drought_resistance, 0).
property(9, earliest_harvest, 19).
property(9, latest_sowing, 31).
property(9, humus_equivalent, -400).
property(9, ground_type, 0).
property(9, ground_type, 1).
property(10, summercrop, 0).
property(10, drought_resistance, 0).
property(10, earliest_harvest, 19).
property(10, latest_sowing, 28).
property(10, humus_equivalent, -400).
property(10, ground_type, 0).
property(10, ground_type, 1).
property(11, summercrop, 0).
property(11, drought_resistance, 1).
property(11, earliest_harvest, 22).
property(11, latest_sowing, 28).
property(11, humus_equivalent, -400).
property(11, ground_type, -1).
property(11, ground_type, 0).
property(12, summercrop, 0).
property(12, drought_resistance, 1).
property(12, earliest_harvest, 19).
property(12, latest_sowing, 28).
property(12, humus_equivalent, -400).
property(12, ground_type, -1).
property(12, ground_type, 0).
property(13, summercrop, 1).
property(13, drought_resistance, -1).
property(13, earliest_harvest, 20).
property(13, latest_sowing, 9).
property(13, humus_equivalent, -400).
property(13, ground_type, 0).
property(14, summercrop, 1).
property(14, drought_resistance, -1).
property(14, earliest_harvest, 21).
property(14, latest_sowing, 8).
property(14, humus_equivalent, -400).
property(14, ground_type, 0).
property(15, summercrop, 1).
property(15, drought_resistance, 1).
property(15, earliest_harvest, 27).
property(15, latest_sowing, 15).
property(15, humus_equivalent, -800).
property(15, ground_type, -1).
property(15, ground_type, 0).
property(16, summercrop, 1).
property(16, drought_resistance, -1).
property(16, earliest_harvest, 25).
property(16, latest_sowing, 13).
property(16, humus_equivalent, -800).
property(16, ground_type, 0).
property(16, ground_type, 1).
property(17, summercrop, 1).
property(17, drought_resistance, -1).
property(17, earliest_harvest, 24).
property(17, latest_sowing, 13).
property(17, humus_equivalent, -800).
property(17, ground_type, 0).
property(17, ground_type, 1).
property(18, summercrop, 1).
property(18, drought_resistance, 0).
property(18, earliest_harvest, 24).
property(18, latest_sowing, 10).
property(18, humus_equivalent, -1300).
property(18, ground_type, 0).
property(19, summercrop, 1).
property(19, drought_resistance, -1).
property(19, earliest_harvest, 22).
property(19, latest_sowing, 14).
property(19, humus_equivalent, -1000).
property(19, ground_type, -1).
property(19, ground_type, 0).
property(20, summercrop, 0).
property(20, drought_resistance, 0).
property(20, earliest_harvest, 19).
property(20, latest_sowing, 24).
property(20, humus_equivalent, -400).
property(20, ground_type, 0).
property(20, ground_type, 1).
property(21, summercrop, 1).
property(21, drought_resistance, -1).
property(21, earliest_harvest, 23).
property(21, latest_sowing, 11).
property(21, humus_equivalent, -400).
property(21, ground_type, -1).
property(21, ground_type, 0).
property(22, summercrop, 1).
property(22, drought_resistance, -1).
property(22, earliest_harvest, 24).
property(22, latest_sowing, 13).
property(22, humus_equivalent, -1000).
property(22, ground_type, -1).
property(22, ground_type, 0).
property(23, summercrop, 0).
property(23, drought_resistance, 1).
property(23, earliest_harvest, 13).
property(23, latest_sowing, 25).
property(23, humus_equivalent, 1300).
property(23, ground_type, -1).
property(23, ground_type, 0).
property(23, ground_type, 1).
cropbreak(0, single, -1).
cropbreak(0, single, -2).
cropbreak(0, single, -3).
cropbreak(0, single, -4).
cropbreak(1, single, -1).
cropbreak(1, single, -2).
cropbreak(1, single, -3).
cropbreak(1, single, -4).
cropbreak(1, single, -5).
cropbreak(2, single, -1).
cropbreak(2, single, -2).
cropbreak(2, single, -3).
cropbreak(3, single, -1).
cropbreak(3, single, -2).
cropbreak(3, single, -3).
cropbreak(3, single, -4).
cropbreak(3, single, -5).
cropbreak(4, single, -1).
cropbreak(4, single, -2).
cropbreak(4, single, -3).
cropbreak(5, single, -1).
cropbreak(5, single, -2).
cropbreak(5, single, -3).
cropbreak(5, single, -4).
cropbreak(6, single, -1).
cropbreak(6, single, -2).
cropbreak(7, single, -1).
cropbreak(7, single, -2).
cropbreak(8, single, -1).
cropbreak(8, single, -2).
cropbreak(9, single, -1).
cropbreak(9, single, -2).
cropbreak(10, single, -1).
cropbreak(10, single, -2).
cropbreak(11, single, -1).
cropbreak(12, single, -1).
cropbreak(12, single, -2).
cropbreak(13, single, -1).
cropbreak(13, single, -2).
cropbreak(14, single, -1).
cropbreak(14, single, -2).
cropbreak(14, single, -3).
cropbreak(14, single, -4).
cropbreak(15, single, -1).
cropbreak(15, single, -2).
cropbreak(16, single, -1).
cropbreak(17, single, -1).
cropbreak(18, single, -1).
cropbreak(18, single, -2).
cropbreak(18, single, -3).
cropbreak(18, single, -4).
cropbreak(19, single, -1).
cropbreak(19, single, -2).
cropbreak(19, single, -3).
cropbreak(20, single, -1).
cropbreak(20, single, -2).
cropbreak(20, single, -3).
cropbreak(21, single, -1).
cropbreak(21, single, -2).
cropbreak(21, single, -3).
cropbreak(22, single, -1).
cropbreak(22, single, -2).

mfgroup((ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(6, (ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(7, (ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(8, (ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(9, (ge;geohnemaishaferhirse)).
crop_mfgroups(10, (ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(11, (ge;geohnemaishaferhirse)).
crop_mfgroups(12, (ge;geohnemaishaferhirse)).
crop_mfgroups(13, (ge;geohnemaishaferhirse)).
crop_mfgroups(14, ge).
crop_mfgroups(15, ge).
crop_mfgroups(17, ge).
mf_group_block(ge, (6;7;8;9;10;11;12;13;14;15;17), -4..-1).
mf_group_block(geohnemaishaferhirse, (6;7;8;9;10;11;12;13), -3..-1).
mf_group_block(weizentriticale, (6;7;8;10), -3..-1).
mf_group_block_max_count(geohnemaishaferhirse, 2).
mf_group_block_max_count(ge, 3).
mf_group_block_max_count(weizentriticale, 1).

apgroups(0, (blatt;fl;l)).
apgroups(1, (blatt;fl;l)).
apgroups(2, (blatt;l)).
apgroups(3, (blatt;l)).
apgroups(4, (blatt;l)).
apgroups(5, (blatt;l)).
apgroups(6, (weizendinkeltriticale;weizen)).
apgroups(7, (weizendinkeltriticale;weizen)).
apgroups(8, (weizendinkeltriticale;weizen)).
apgroups(9, weizendinkeltriticale).
apgroups(10, weizendinkeltriticale).
apgroups(12, gerste).
apgroups(13, gerste).
apgroups(16, (blatt;mais)).
apgroups(17, mais).
apgroups(18, (blatt;ruebenkruziferen)).
apgroups(19, blatt).
apgroups(20, (rapssonnenblume;ruebenkruziferen)).
apgroups(21, rapssonnenblume).
ap_group_block(blatt, (0;1;2;3;4;5;16;18;19), -1).
ap_group_block(fl, (0;1), -5..-1).
ap_group_block(l, (0;1;2;3;4;5), -4..-1).
ap_group_block(weizendinkeltriticale, (6;7;8;9;10), -1).
ap_group_block(rapssonnenblume, (20;21), -2..-1).
ap_group_block(mais, (16;17), -1).
ap_group_block(ruebenkruziferen, (18;20), -3..-1).
ap_group_block(weizen, (18;20), -2..-1).
ap_group_block(gerste, (12;13), -2..-1).

% Current configuration
ground_type_info(0).
drywet_info(0).
week_info(24).

previous_actions_info(-5,6).
previous_actions_info(-4,1).
previous_actions_info(-3,8).
previous_actions_info(-2,9).
previous_actions_info(-1,1).

% Define
% Create AP Group filters
blocked_by_previous(A) :- action(A), previous_actions_info(X,A), cropbreak(A, single, X).
blocked_by_ap_group(A) :- apgroups(A,APG), previous_actions_info(X,Y), ap_group_block(APG, Y, X).

% Create MF Group filters
mf_group_block_active(MFG, Y) :- mf_group_block(MFG, A, Y), previous_actions_info(Y,A).
count_mf_group(MFG, C) :- C = #count {Y : mf_group_block_active(MFG, Y)}, mfgroup(MFG).
blocked_by_mf_group(A) :- action(A), crop_mfgroups(A, MFG), count_mf_group(MFG, C), mf_group_block_max_count(MFG, MC), C+1 > MC.

% Create filters for properties
blocked_by_week(A) :- action(A), property(A, latest_sowing, LS), week_info(W),  W > LS-1, property(A, summercrop, SC), SC == 0.
allowed_by_drywet(A) :- action(A), property(A, drought_resistance, DR), drywet_info(DW),  DR > -1, DW == 0.
allowed_by_drywet(A) :- action(A), drywet_info(DW), DW == 1.
allowed_by_groundtype(A) :- action(A), ground_type_info(GT), property(A, ground_type, GT).

% Filter solution candidates
immediate_candidate(A) :- action(A), not blocked_by_previous(A), not blocked_by_ap_group(A), not blocked_by_mf_group(A), not blocked_by_week(A), allowed_by_drywet(A), allowed_by_groundtype(A).

% Define the output predicate
#show immediate_candidate/1.
"""

# Load the Clingo program
control = clingo.Control()
control.add("base", [], program)
control.ground([("base", [])])

# Solve the program and print the immediate candidate actions
solutions = []
def on_model(model):
    solutions.append(model.symbols(shown=True))

control.solve(on_model=on_model)
for solution in solutions:
    # print(solution)
    for symbol in solution:
        # print(symbol.arguments[0])
        print(symbol)

immediate_candidate(15)


# Filter check

In [28]:
import clingo 
import numpy as np
# Define the Clingo program as a string
program_start = """
% Define the input data as facts
previous_time(-5..-1).
action(0..23).
property(0, summercrop, 0).
property(0, drought_resistance, 0).
property(0, earliest_harvest, 13).
property(0, latest_sowing, 25).
property(0, humus_equivalent, 1300).
property(0, ground_type, 0).
property(1, summercrop, 0).
property(1, drought_resistance, 1).
property(1, earliest_harvest, 13).
property(1, latest_sowing, 22).
property(1, humus_equivalent, 1300).
property(1, ground_type, -1).
property(1, ground_type, 0).
property(2, summercrop, 1).
property(2, drought_resistance, 0).
property(2, earliest_harvest, 19).
property(2, latest_sowing, 9).
property(2, humus_equivalent, 160).
property(2, ground_type, 0).
property(2, ground_type, 1).
property(3, summercrop, 1).
property(3, drought_resistance, -1).
property(3, earliest_harvest, 19).
property(3, latest_sowing, 11).
property(3, humus_equivalent, 160).
property(3, ground_type, 0).
property(3, ground_type, 1).
property(4, summercrop, 1).
property(4, drought_resistance, -1).
property(4, earliest_harvest, 25).
property(4, latest_sowing, 12).
property(4, humus_equivalent, 160).
property(4, ground_type, 0).
property(5, summercrop, 1).
property(5, drought_resistance, 0).
property(5, earliest_harvest, 23).
property(5, latest_sowing, 9).
property(5, humus_equivalent, 160).
property(5, ground_type, 0).
property(6, summercrop, 0).
property(6, drought_resistance, 0).
property(6, earliest_harvest, 19).
property(6, latest_sowing, 31).
property(6, humus_equivalent, -400).
property(6, ground_type, 0).
property(6, ground_type, 1).
property(7, summercrop, 1).
property(7, drought_resistance, -1).
property(7, earliest_harvest, 20).
property(7, latest_sowing, 8).
property(7, humus_equivalent, -400).
property(7, ground_type, 0).
property(7, ground_type, 1).
property(8, summercrop, 0).
property(8, drought_resistance, 0).
property(8, earliest_harvest, 19).
property(8, latest_sowing, 28).
property(8, humus_equivalent, -400).
property(8, ground_type, 0).
property(8, ground_type, 1).
property(9, summercrop, 0).
property(9, drought_resistance, 0).
property(9, earliest_harvest, 19).
property(9, latest_sowing, 31).
property(9, humus_equivalent, -400).
property(9, ground_type, 0).
property(9, ground_type, 1).
property(10, summercrop, 0).
property(10, drought_resistance, 0).
property(10, earliest_harvest, 19).
property(10, latest_sowing, 28).
property(10, humus_equivalent, -400).
property(10, ground_type, 0).
property(10, ground_type, 1).
property(11, summercrop, 0).
property(11, drought_resistance, 1).
property(11, earliest_harvest, 22).
property(11, latest_sowing, 28).
property(11, humus_equivalent, -400).
property(11, ground_type, -1).
property(11, ground_type, 0).
property(12, summercrop, 0).
property(12, drought_resistance, 1).
property(12, earliest_harvest, 19).
property(12, latest_sowing, 28).
property(12, humus_equivalent, -400).
property(12, ground_type, -1).
property(12, ground_type, 0).
property(13, summercrop, 1).
property(13, drought_resistance, -1).
property(13, earliest_harvest, 20).
property(13, latest_sowing, 9).
property(13, humus_equivalent, -400).
property(13, ground_type, 0).
property(14, summercrop, 1).
property(14, drought_resistance, -1).
property(14, earliest_harvest, 21).
property(14, latest_sowing, 8).
property(14, humus_equivalent, -400).
property(14, ground_type, 0).
property(15, summercrop, 1).
property(15, drought_resistance, 1).
property(15, earliest_harvest, 27).
property(15, latest_sowing, 15).
property(15, humus_equivalent, -800).
property(15, ground_type, -1).
property(15, ground_type, 0).
property(16, summercrop, 1).
property(16, drought_resistance, -1).
property(16, earliest_harvest, 25).
property(16, latest_sowing, 13).
property(16, humus_equivalent, -800).
property(16, ground_type, 0).
property(16, ground_type, 1).
property(17, summercrop, 1).
property(17, drought_resistance, -1).
property(17, earliest_harvest, 24).
property(17, latest_sowing, 13).
property(17, humus_equivalent, -800).
property(17, ground_type, 0).
property(17, ground_type, 1).
property(18, summercrop, 1).
property(18, drought_resistance, 0).
property(18, earliest_harvest, 24).
property(18, latest_sowing, 10).
property(18, humus_equivalent, -1300).
property(18, ground_type, 0).
property(19, summercrop, 1).
property(19, drought_resistance, -1).
property(19, earliest_harvest, 22).
property(19, latest_sowing, 14).
property(19, humus_equivalent, -1000).
property(19, ground_type, -1).
property(19, ground_type, 0).
property(20, summercrop, 0).
property(20, drought_resistance, 0).
property(20, earliest_harvest, 19).
property(20, latest_sowing, 24).
property(20, humus_equivalent, -400).
property(20, ground_type, 0).
property(20, ground_type, 1).
property(21, summercrop, 1).
property(21, drought_resistance, -1).
property(21, earliest_harvest, 23).
property(21, latest_sowing, 11).
property(21, humus_equivalent, -400).
property(21, ground_type, -1).
property(21, ground_type, 0).
property(22, summercrop, 1).
property(22, drought_resistance, -1).
property(22, earliest_harvest, 24).
property(22, latest_sowing, 13).
property(22, humus_equivalent, -1000).
property(22, ground_type, -1).
property(22, ground_type, 0).
property(23, summercrop, 0).
property(23, drought_resistance, 1).
property(23, earliest_harvest, 13).
property(23, latest_sowing, 25).
property(23, humus_equivalent, 1300).
property(23, ground_type, -1).
property(23, ground_type, 0).
property(23, ground_type, 1).
cropbreak(0, single, -1).
cropbreak(0, single, -2).
cropbreak(0, single, -3).
cropbreak(0, single, -4).
cropbreak(1, single, -1).
cropbreak(1, single, -2).
cropbreak(1, single, -3).
cropbreak(1, single, -4).
cropbreak(1, single, -5).
cropbreak(2, single, -1).
cropbreak(2, single, -2).
cropbreak(2, single, -3).
cropbreak(3, single, -1).
cropbreak(3, single, -2).
cropbreak(3, single, -3).
cropbreak(3, single, -4).
cropbreak(3, single, -5).
cropbreak(4, single, -1).
cropbreak(4, single, -2).
cropbreak(4, single, -3).
cropbreak(5, single, -1).
cropbreak(5, single, -2).
cropbreak(5, single, -3).
cropbreak(5, single, -4).
cropbreak(6, single, -1).
cropbreak(6, single, -2).
cropbreak(7, single, -1).
cropbreak(7, single, -2).
cropbreak(8, single, -1).
cropbreak(8, single, -2).
cropbreak(9, single, -1).
cropbreak(9, single, -2).
cropbreak(10, single, -1).
cropbreak(10, single, -2).
cropbreak(11, single, -1).
cropbreak(12, single, -1).
cropbreak(12, single, -2).
cropbreak(13, single, -1).
cropbreak(13, single, -2).
cropbreak(14, single, -1).
cropbreak(14, single, -2).
cropbreak(14, single, -3).
cropbreak(14, single, -4).
cropbreak(15, single, -1).
cropbreak(15, single, -2).
cropbreak(16, single, -1).
cropbreak(17, single, -1).
cropbreak(18, single, -1).
cropbreak(18, single, -2).
cropbreak(18, single, -3).
cropbreak(18, single, -4).
cropbreak(19, single, -1).
cropbreak(19, single, -2).
cropbreak(19, single, -3).
cropbreak(20, single, -1).
cropbreak(20, single, -2).
cropbreak(20, single, -3).
cropbreak(21, single, -1).
cropbreak(21, single, -2).
cropbreak(21, single, -3).
cropbreak(22, single, -1).
cropbreak(22, single, -2).

mfgroup((ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(6, (ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(7, (ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(8, (ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(9, (ge;geohnemaishaferhirse)).
crop_mfgroups(10, (ge;geohnemaishaferhirse;weizentriticale)).
crop_mfgroups(11, (ge;geohnemaishaferhirse)).
crop_mfgroups(12, (ge;geohnemaishaferhirse)).
crop_mfgroups(13, (ge;geohnemaishaferhirse)).
crop_mfgroups(14, ge).
crop_mfgroups(15, ge).
crop_mfgroups(17, ge).
mf_group_block(ge, (6;7;8;9;10;11;12;13;14;15;17), -4..-1).
mf_group_block(geohnemaishaferhirse, (6;7;8;9;10;11;12;13), -3..-1).
mf_group_block(weizentriticale, (6;7;8;10), -3..-1).
mf_group_block_max_count(geohnemaishaferhirse, 2).
mf_group_block_max_count(ge, 3).
mf_group_block_max_count(weizentriticale, 1).

apgroups(0, (blatt;fl;l)).
apgroups(1, (blatt;fl;l)).
apgroups(2, (blatt;l)).
apgroups(3, (blatt;l)).
apgroups(4, (blatt;l)).
apgroups(5, (blatt;l)).
apgroups(6, (weizendinkeltriticale;weizen)).
apgroups(7, (weizendinkeltriticale;weizen)).
apgroups(8, (weizendinkeltriticale;weizen)).
apgroups(9, weizendinkeltriticale).
apgroups(10, weizendinkeltriticale).
apgroups(12, gerste).
apgroups(13, gerste).
apgroups(16, (blatt;mais)).
apgroups(17, mais).
apgroups(18, (blatt;ruebenkruziferen)).
apgroups(19, blatt).
apgroups(20, (rapssonnenblume;ruebenkruziferen)).
apgroups(21, rapssonnenblume).
ap_group_block(blatt, (0;1;2;3;4;5;16;18;19), -1).
ap_group_block(fl, (0;1), -5..-1).
ap_group_block(l, (0;1;2;3;4;5), -4..-1).
ap_group_block(weizendinkeltriticale, (6;7;8;9;10), -1).
ap_group_block(rapssonnenblume, (20;21), -2..-1).
ap_group_block(mais, (16;17), -1).
ap_group_block(ruebenkruziferen, (18;20), -3..-1).
ap_group_block(weizen, (18;20), -2..-1).
ap_group_block(gerste, (12;13), -2..-1).

% Current configuration
"""

program_end = """
% Define
% Create AP Group filters
blocked_by_previous(A) :- action(A), previous_actions_info(X,A), cropbreak(A, single, X).
blocked_by_ap_group(A) :- apgroups(A,APG), previous_actions_info(X,Y), ap_group_block(APG, Y, X).

% Create MF Group filters
mf_group_block_active(MFG, Y) :- mf_group_block(MFG, A, Y), previous_actions_info(Y,A).
count_mf_group(MFG, C) :- C = #count {Y : mf_group_block_active(MFG, Y)}, mfgroup(MFG).
blocked_by_mf_group(A) :- action(A), crop_mfgroups(A, MFG), count_mf_group(MFG, C), mf_group_block_max_count(MFG, MC), C+1 > MC.

% Create filters for properties
blocked_by_week(A) :- action(A), property(A, latest_sowing, LS), week_info(W),  W > LS-1, property(A, summercrop, SC), SC == 0.
blocked_by_humus(A) :- action(A), property(A, humus_equivalent, HE), humus_info(HL, HML), HL+HE <= HML, HE<0.

% Filter solution candidates
immediate_candidate(A) :- action(A), not blocked_by_previous(A), not blocked_by_ap_group(A), not blocked_by_mf_group(A), not blocked_by_week(A), not blocked_by_humus(A).

% Define the output predicate
#show immediate_candidate/1.
"""


filter_information = np.array([13.0, 1.0, 0.0, 5.992888624849152, 3.0, 9, 12, 14, 0, 0], dtype=object)
week = filter_information[0]
ground_type = filter_information[1]
drywet = filter_information[2]
humus_level = int(filter_information[3]*1000)
humus_minimum_level = int(filter_information[4]*1000)
previous_crops_selected = filter_information[5:]
configuration_string = f"""
week_info({int(week)}).
ground_type_info({int(ground_type)}).
drywet_info({int(drywet)}).
humus_info({humus_level},{humus_minimum_level}).
"""
for i, crop in enumerate(previous_crops_selected):
    flag = False
    if crop is not None and crop != -1.0:
        flag = True
        configuration_string += f"""previous_actions_info({i-5},{int(crop)}).\n"""
    if not flag:
        configuration_string += f"""previous_actions_info({-1},{-1}).\n"""
program = program_start + configuration_string + program_end
# print(program)
# Load the Clingo program
control = clingo.Control()
control.add("base", [], program)
control.ground([("base", [])])

# Solve the program and print the immediate candidate actions
solutions = []
def on_model(model):
    solutions.append(model.symbols(shown=True))

control.solve(on_model=on_model)
for solution in solutions:
    # print(solution)
    print([symbol.arguments[0].number for symbol in solution])
print(configuration_string)

[6, 7, 8, 9, 10, 11, 12, 13, 15, 17, 20, 21, 22, 23]

week_info(13).
ground_type_info(1).
drywet_info(0).
humus_info(5992,3000).
previous_actions_info(-5,9).
previous_actions_info(-4,12).
previous_actions_info(-3,14).
previous_actions_info(-2,0).
previous_actions_info(-1,0).



## Humus Check

In [9]:
import clingo 
import numpy as np
# Define the Clingo program as a string
program= """
% Define the input data as facts
action(0..22).
previous_time(-5..-1).
% Define the input data as facts
action(0..22).
property(0, summercrop, 0).
property(0, drought_resistance, 0).
property(0, earliest_harvest, 13).
property(0, latest_sowing, 25).
property(0, humus_equivalent, 1300).
property(0, ground_type, 0).
property(1, summercrop, 0).
property(1, drought_resistance, 1).
property(1, earliest_harvest, 13).
property(1, latest_sowing, 22).
property(1, humus_equivalent, 1300).
property(1, ground_type, -1).
property(1, ground_type, 0).
property(2, summercrop, 1).
property(2, drought_resistance, 0).
property(2, earliest_harvest, 19).
property(2, latest_sowing, 9).
property(2, humus_equivalent, 160).
property(2, ground_type, 0).
property(2, ground_type, 1).
property(3, summercrop, 1).
property(3, drought_resistance, -1).
property(3, earliest_harvest, 19).
property(3, latest_sowing, 11).
property(3, humus_equivalent, 160).
property(3, ground_type, 0).
property(3, ground_type, 1).
property(4, summercrop, 1).
property(4, drought_resistance, -1).
property(4, earliest_harvest, 25).
property(4, latest_sowing, 12).
property(4, humus_equivalent, 160).
property(4, ground_type, 0).
property(5, summercrop, 1).
property(5, drought_resistance, 0).
property(5, earliest_harvest, 23).
property(5, latest_sowing, 9).
property(5, humus_equivalent, 160).
property(5, ground_type, 0).
property(6, summercrop, 0).
property(6, drought_resistance, 0).
property(6, earliest_harvest, 19).
property(6, latest_sowing, 31).
property(6, humus_equivalent, -400).
property(6, ground_type, 0).
property(6, ground_type, 1).
property(7, summercrop, 1).
property(7, drought_resistance, -1).
property(7, earliest_harvest, 20).
property(7, latest_sowing, 8).
property(7, humus_equivalent, -400).
property(7, ground_type, 0).
property(7, ground_type, 1).
property(8, summercrop, 0).
property(8, drought_resistance, 0).
property(8, earliest_harvest, 19).
property(8, latest_sowing, 28).
property(8, humus_equivalent, -400).
property(8, ground_type, 0).
property(8, ground_type, 1).
property(9, summercrop, 0).
property(9, drought_resistance, 0).
property(9, earliest_harvest, 19).
property(9, latest_sowing, 31).
property(9, humus_equivalent, -400).
property(9, ground_type, 0).
property(9, ground_type, 1).
property(10, summercrop, 0).
property(10, drought_resistance, 0).
property(10, earliest_harvest, 19).
property(10, latest_sowing, 28).
property(10, humus_equivalent, -400).
property(10, ground_type, 0).
property(10, ground_type, 1).
property(11, summercrop, 0).
property(11, drought_resistance, 1).
property(11, earliest_harvest, 22).
property(11, latest_sowing, 28).
property(11, humus_equivalent, -400).
property(11, ground_type, -1).
property(11, ground_type, 0).
property(12, summercrop, 0).
property(12, drought_resistance, 1).
property(12, earliest_harvest, 19).
property(12, latest_sowing, 28).
property(12, humus_equivalent, -400).
property(12, ground_type, -1).
property(12, ground_type, 0).
property(13, summercrop, 1).
property(13, drought_resistance, -1).
property(13, earliest_harvest, 20).
property(13, latest_sowing, 9).
property(13, humus_equivalent, -400).
property(13, ground_type, 0).
property(14, summercrop, 1).
property(14, drought_resistance, -1).
property(14, earliest_harvest, 21).
property(14, latest_sowing, 8).
property(14, humus_equivalent, -400).
property(14, ground_type, 0).
property(15, summercrop, 1).
property(15, drought_resistance, 1).
property(15, earliest_harvest, 27).
property(15, latest_sowing, 15).
property(15, humus_equivalent, -800).
property(15, ground_type, -1).
property(15, ground_type, 0).
property(16, summercrop, 1).
property(16, drought_resistance, -1).
property(16, earliest_harvest, 25).
property(16, latest_sowing, 13).
property(16, humus_equivalent, -800).
property(16, ground_type, 0).
property(16, ground_type, 1).
property(17, summercrop, 1).
property(17, drought_resistance, -1).
property(17, earliest_harvest, 24).
property(17, latest_sowing, 13).
property(17, humus_equivalent, -800).
property(17, ground_type, 0).
property(17, ground_type, 1).
property(18, summercrop, 1).
property(18, drought_resistance, 0).
property(18, earliest_harvest, 24).
property(18, latest_sowing, 10).
property(18, humus_equivalent, -1300).
property(18, ground_type, 0).
property(19, summercrop, 1).
property(19, drought_resistance, -1).
property(19, earliest_harvest, 22).
property(19, latest_sowing, 14).
property(19, humus_equivalent, -1000).
property(19, ground_type, -1).
property(19, ground_type, 0).
property(20, summercrop, 0).
property(20, drought_resistance, 0).
property(20, earliest_harvest, 19).
property(20, latest_sowing, 24).
property(20, humus_equivalent, -400).
property(20, ground_type, 0).
property(20, ground_type, 1).
property(21, summercrop, 1).
property(21, drought_resistance, -1).
property(21, earliest_harvest, 23).
property(21, latest_sowing, 11).
property(21, humus_equivalent, -400).
property(21, ground_type, -1).
property(21, ground_type, 0).
property(22, summercrop, 1).
property(22, drought_resistance, -1).
property(22, earliest_harvest, 24).
property(22, latest_sowing, 13).
property(22, humus_equivalent, -1000).
property(22, ground_type, -1).
property(22, ground_type, 0).

% Current configuration
humus_equivalent_info(2450,2000).

% Define

% Create filters for properties
blocked_by_humus(A) :- action(A), property(A, humus_equivalent, HE), humus_equivalent_info(HL, HML), HL+HE <= HML, HE<0.

% Filter solution candidates
immediate_candidate(A) :- action(A), not blocked_by_humus(A).

% Define the output predicate
#show immediate_candidate/1.
"""


# Load the Clingo program
control = clingo.Control()
control.add("base", [], program)
control.ground([("base", [])])

# Solve the program and print the immediate candidate actions
solutions = []
def on_model(model):
    solutions.append(model.symbols(shown=True))

control.solve(on_model=on_model)
for solution in solutions:
    # print(solution)
    print([symbol.arguments[0].number for symbol in solution])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 20, 21]
